# Optiver Trading At The Close

[Competition](https://www.kaggle.com/competitions/optiver-trading-at-the-close)

[GitHub Repository](https://github.com/cvaisnor/DNN-Final-Project)

[Google Slides Presentation](https://docs.google.com/presentation/d/1Xc5F1_NveFi1il3GqHej2aqVmklR_jZU4kx6ZJhiDEM/edit?usp=sharing)

# General Imports

In [1]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

kaggle_environment = False # True if running on Kaggle, don't forget to add the dataset!

if kaggle_environment:
    data_path = '/kaggle/input/'
else:
    data_path = 'kaggle/input/'

# disable PkgResourcesDeprecationWarning
import warnings
warnings.filterwarnings('ignore')

# Setting File Structure

In [2]:
for dirname, _, filenames in os.walk(data_path):
    for filename in filenames:
        print(os.path.join(dirname, filename))

kaggle/input/optiver-trading-at-the-close/public_timeseries_testing_util.py
kaggle/input/optiver-trading-at-the-close/.gitkeep
kaggle/input/optiver-trading-at-the-close/train.csv
kaggle/input/optiver-trading-at-the-close/optiver2023/__init__.py
kaggle/input/optiver-trading-at-the-close/optiver2023/competition.cpython-310-x86_64-linux-gnu.so
kaggle/input/optiver-trading-at-the-close/optiver2023/__pycache__/__init__.cpython-310.pyc
kaggle/input/optiver-trading-at-the-close/example_test_files/test.csv
kaggle/input/optiver-trading-at-the-close/example_test_files/revealed_targets.csv
kaggle/input/optiver-trading-at-the-close/example_test_files/sample_submission.csv


# Loading Data

In [3]:
train = pd.read_csv(data_path + 'optiver-trading-at-the-close/train.csv')
revealed_targets = pd.read_csv(data_path + 'optiver-trading-at-the-close/example_test_files/revealed_targets.csv')
test = pd.read_csv(data_path + 'optiver-trading-at-the-close/example_test_files/test.csv')
sample_submission = pd.read_csv(data_path + 'optiver-trading-at-the-close/example_test_files/sample_submission.csv')

# Feature Engineering Section

In [4]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

class CustomIterativeImputer(BaseEstimator, TransformerMixin):
    def __init__(self, columns, max_iter=10, random_state=0):
        self.columns = columns
        self.max_iter = max_iter
        self.random_state = random_state
        self.imputer = None

    def fit(self, X, y=None):
        # Initialize the IterativeImputer
        self.imputer = IterativeImputer(max_iter=self.max_iter, random_state=self.random_state)
        self.imputer.fit(X[self.columns])
        return self

    def transform(self, X):
        # Check is fit had been called
        if self.imputer is None:
            raise RuntimeError("You must call fit before calling transform")
        
        # Fit the imputer and transform the data
        X[self.columns] = self.imputer.transform(X[self.columns])
        return X

# Create the pipeline with the custom imputer
pipeline = Pipeline([
    ('iterative_imputer', CustomIterativeImputer(columns=['far_price', 'near_price'])),
    # Add other steps here if necessary
])

In [5]:
# Use the pipeline on the training data
df_processed = pipeline.fit_transform(train)

# drop all rows with NaNs (500-ish rows after imputation)
df_processed = df_processed.dropna()

# drop row_id column
df_processed = df_processed.drop(columns=['row_id'])

In [6]:
# def add_datetime_feature(df):
#     """
#     Add datetime features to a DataFrame.

#     Parameters:
#     df (pandas.DataFrame): DataFrame to process.

#     Returns:
#     pandas.DataFrame: DataFrame with added datetime feature.
#     """
#     df = df.copy()
#     start_time = pd.to_timedelta('9:30:00')
#     # Convert date_id to actual dates starting from 2020-01-01
#     df['date'] = pd.to_datetime('2020-01-01') + pd.to_timedelta(df['date_id'], unit='D')
#     df['time'] = pd.to_timedelta(df['time_id'], unit='s')
#     # Add the trading start time to the time column
#     df['time'] = df['time'] + start_time
#     # Create long-form datetime timestamp
#     df['timestamp'] = df['date'] + df['time']
#     df.drop(['date', 'time', 'date_id'], axis=1, inplace=True)

#     # add day
#     df['day'] = df['timestamp'].dt.day

#     # Add unix time
#     df['unix_time'] = df['timestamp'].astype(np.int64) // 10**9

#     # Add a time_idx (an sequence of consecutive integers that goes from min to max date)
#     df['time_idx'] = df['timestamp'].rank(method='dense').astype(int) - 1

#     return df

In [7]:
# df_with_datetime = add_datetime_feature(df_processed)

In [8]:
def generate_features(test):
    return test

# Model Development

In [9]:
import os
import warnings

warnings.filterwarnings("ignore")  # avoid printing out absolute paths

In [10]:
import copy
from pathlib import Path
import warnings

import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor
from lightning.pytorch.loggers import TensorBoardLogger
import numpy as np
import pandas as pd
import torch

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import MAE, SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

2023-12-06 14:27:40.891533: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-06 14:27:40.909672: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX512F AVX512_VNNI AVX512_BF16, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [25]:
# print pytorch version
print(f"Using torch {torch.__version__}")

# print pytorch lightning version
print(f"Using pytorch lightning {pl.__version__}")

Using torch 2.1.1+cu121
Using pytorch lightning 2.1.2


In [11]:
# we want to predict 1 minute ahead of each stock_id
max_prediction_length = 60 # (seconds)

# max encoder length = number of seconds in 7 hours
max_encoder_length = 7 * 60 * 60 # (seconds)

In [13]:
# convert "stock_id" to string so it is categorical
df_processed['stock_id'] = df_processed['stock_id'].astype(str)

In [17]:
# creating the dataset
training = TimeSeriesDataSet(
    df_processed,
    time_idx="time_id",
    target="target",
    group_ids=["stock_id"],
    min_encoder_length=0,  # allow predictions without history
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    static_categoricals=["stock_id"],
    time_varying_unknown_reals=["target"],
    allow_missing_timesteps=True,
)

In [19]:
# TypeError: Addition/subtraction of integers and integer-arrays with Timestamp is no longer supported.  Instead of adding/subtracting `n`, use `n * obj.freq`

# create validation set (predict=True) which means to predict the last max_prediction_length points in time for each series
validation = TimeSeriesDataSet.from_dataset(training, df_processed, predict=True, stop_randomization=True)

In [20]:
# creating dataloaders
batch_size = 128  # set this between 32 to 128
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)

In [21]:
# calculate baseline mean absolute error, i.e. predict next value as the last available value from the history
baseline_predictions = Baseline().predict(val_dataloader, return_y=True)
MAE()(baseline_predictions.output, baseline_predictions.y)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


tensor(6.0678, device='cuda:0')

In [22]:
PATIENCE = 30
MAX_EPOCHS = 120
LEARNING_RATE = 0.03
OPTUNA = False

# configure network and trainer
pl.seed_everything(42)
trainer = pl.Trainer(
    accelerator="gpu",
    # clipping gradients is a hyperparameter and important to prevent divergance
    # of the gradient for recurrent neural networks
    gradient_clip_val=0.1,
)

tft = TemporalFusionTransformer.from_dataset(
    training,
    # not meaningful for finding the learning rate but otherwise very important
    learning_rate=0.03,
    hidden_size=8,  # most important hyperparameter apart from learning rate
    # number of attention heads. Set to up to 4 for large datasets
    attention_head_size=1,
    dropout=0.1,  # between 0.1 and 0.3 are good values
    hidden_continuous_size=8,  # set to <= hidden_size
    loss=QuantileLoss(),
    optimizer="Ranger",
    # reduce learning rate if no improvement in validation loss after x epochs
    reduce_on_plateau_patience=1000,
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Number of parameters in network: 6.2k


In [23]:
# find optimal learning rate
from lightning.pytorch.tuner import Tuner

res = Tuner(trainer).lr_find(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
    max_lr=10.0,
    min_lr=1e-6,
)

print(f"suggested learning rate: {res.suggestion()}")
fig = res.plot(show=True, suggest=True)
fig.show()

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

AttributeError: 'LearningRateFinder' object has no attribute 'optimal_lr'

In [ ]:
tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=LEARNING_RATE,
    hidden_size=16,  # biggest influence network size
    attention_head_size=1,
    dropout=0.1,
    hidden_continuous_size=8,
    output_size=7,  # QuantileLoss has 7 quantiles by default
    loss=QuantileLoss(),
    # reduce learning rate if no improvement in validation loss after x epochs
    reduce_on_plateau_patience=4,
)

print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

In [ ]:
# fit network
trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

# Generating Predictions

Only for Kaggle:
When uploading notebook, make sure to add the Optiver data on the right side of the screen.

In [ ]:
if kaggle_environment:
    import optiver2023
    env = optiver2023.make_env()
    iter_test = env.iter_test()

In [ ]:
if kaggle_environment:
    # To count how many time the "for loop" runs.
    counter = 0

    # init 3 empty lists
    test_ls, revealed_targets_ls, sample_prediction_ls = [], [], []

    for (test, revealed_targets, sample_prediction) in iter_test:
        # Append the dataframe that API return into the list.
        test_ls.append(test.copy())
        revealed_targets_ls.append(revealed_targets.copy())
        sample_prediction_ls.append(sample_prediction.copy())

        # Generate features
        test = generate_features(test)

        # Writes our predictions 
        sample_prediction["target"] = model.predict(test)
        
        # This line submit our predictions.
        env.predict(sample_prediction)
        counter += 1

    print('\n', '=' * 50, sep="")
    print(f"counter: {counter}")